In [ ]:
!pip install -q together
!pip install -q FlagEmbedding
!pip install -q peft
!pip install -q faiss-gpu

In [ ]:
from google.colab import userdata
TOGETHER_API_KEY = userdata.get('TOGETHER_API_KEY')

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from FlagEmbedding import FlagModel
from FlagEmbedding import FlagReranker

import os
import torch
import faiss
import re
import json
import numpy as np
import requests

from together import Together

In [ ]:
together = Together(api_key=TOGETHER_API_KEY)

In [ ]:
# encode_queries()和encode()的区别：
# for s2p (short query to long passage) retrieval task, suggest to use encode_queries() which will automatically add the instruction to each query
# corpus in retrieval task can still use encode() or encode_corpus(), since they don't need instruction

model = FlagModel('BAAI/bge-base-en-v1.5', query_instruction_for_retrieval="Represent this sentence for searching relevant passages:", use_fp16=True)
# print(model.device)

reranker = FlagReranker('BAAI/bge-reranker-base', use_fp16=True, device='cuda')
# print(reranker.device)

In [ ]:
# query = "Tell me the exam information for the database class."
query = "what is happening in Los Angeles?"

courses_numbers = ['544', '566', '585', '596', '599', '626', '677', '699']
course_names = ['Applied Natural Language Processing (NLP)', 'Deep Learning and its Applications (DL)', 'Database Systems (database)', 'Scientific Computing and Visualization',
         'Distributed Systems', 'Text as Data', 'Advanced Computer Vision (CV)', 'Robotic Perception (Robotics)']

full_course_info = dict(zip(['CSCI' + num for num in courses_numbers], ['CSCI' + num + ' ' + name for num, name in zip(courses_numbers, course_names)]))

name_to_num = dict(zip(course_names, courses_numbers))

In [ ]:
full_course_info

{'CSCI544': 'CSCI544 Applied Natural Language Processing (NLP)',
 'CSCI566': 'CSCI566 Deep Learning and its Applications (DL)',
 'CSCI585': 'CSCI585 Database Systems (database)',
 'CSCI596': 'CSCI596 Scientific Computing and Visualization',
 'CSCI599': 'CSCI599 Distributed Systems',
 'CSCI626': 'CSCI626 Text as Data',
 'CSCI677': 'CSCI677 Advanced Computer Vision (CV)',
 'CSCI699': 'CSCI699 Robotic Perception (Robotics)'}

In [ ]:
def build_vector_database(course_names):
    course_embeddings = model.encode(course_names)
    np_course_embeddings = np.array(course_embeddings).astype('float32')

    # with normalization -> bacome cosine similarity search
    faiss.normalize_L2(np_course_embeddings)

    # IndexFlatIP: dot-product search, IndexFlatL2: L2 search
    index_innerproduct = faiss.IndexFlatIP(len(course_embeddings[0]))
    res = faiss.StandardGpuResources()  # Create GPU resources
    gpu_index = faiss.index_cpu_to_gpu(res, 0, index_innerproduct) # Move index to GPU

    gpu_index.add(np_course_embeddings)

    return gpu_index

database = build_vector_database(course_names)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
def search_(query, database, embed_dim, topk=1):
    query_embed = model.encode_queries(query)
    query_embed = np.array(query_embed).astype('float32')

    _, idx = database.search(query_embed.reshape((1, embed_dim)), topk) # (768,) -> (1, 768)
    idx = idx.reshape(-1)

    ret = [course_names[i] for i in idx]

    return ret

def find_syllabus(query, courses_numbers, course_names, name_to_num, model, database):
    def find_course():
        for num in courses_numbers:
            if num in query:
                return num

        result = search_(query, database, embed_dim=database.d, topk=1)
        return result[0]

    result = find_course()

    if result in courses_numbers:
        return 'CSCI' + result + '.txt'
    else:
        return 'CSCI' + name_to_num[result] + '.txt'

In [ ]:
print(find_syllabus(query, courses_numbers, course_names, name_to_num, model, database))

CSCI596.txt


In [ ]:
# 读取txt文件并按"\n\n"切割 -> 按照需求自行修改！！！
def read_and_split_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    segments = content.split('##')
    # print(segments)

    return [seg.strip() for seg in segments]  # 变成了一个list

# 计算query和每个文档分段的相似度，并返回排序后的结果
def compute_similarity(query, segments):
    # encode_queries是专门用于encode query的
    query_embedding = model.encode_queries(query)  # [512, ]
    segment_embeddings = model.encode(segments)  # [num_seg, 512]

    # 官方提供的。只计算点积，不计算cos-simi
    similarities = [query_embedding @ segment_embedding.T for segment_embedding in segment_embeddings]

    # 将分数、query和文档分段组合并排序 -> 降序
    results = sorted(zip(similarities, segments), key=lambda x: x[0], reverse=True)
    # print(results)

    return results

# 定义生成查询和文档组合的函数
def generate_query_passage_pairs(query, passages):
    return [[query[0], passage] for passage in passages]

In [ ]:
def RAG(query):

    if query is None:
        raise ValueError('Your query cannot be empty!')

    k = 3

    final_syllabus = find_syllabus(query, courses_numbers, course_names, name_to_num, model, database)
    print(final_syllabus)

    # file_path = '/content' + os.sep + final_syllabus
    file_path = '/content/drive/MyDrive/CSCI544/Project/syllabus/' + final_syllabus
    knowledge_base = read_and_split_file(file_path)

    query = [query]
    all_similarity = compute_similarity(query, knowledge_base)

    results = []
    for score, segment in all_similarity:
        if score > 0.25:
            results.append(segment)

    query_passage_pairs = generate_query_passage_pairs(query, results)

    if query_passage_pairs:
        scores = reranker.compute_score(query_passage_pairs)
        sorted_query_passage_pairs = sorted(zip(scores, query_passage_pairs), key=lambda x: x[0], reverse=True)

        sorted_results = [(score, pair[1]) for score, pair in sorted_query_passage_pairs]

        final_results = []
        for score, passage in sorted_results:
            # if score > 0:
            final_results.append((score, passage))

        final_results = final_results[:k]
    else:
        final_results = [(0.0, 'There is no relevant information for the given query!')]

    all_segments = '\n\n'.join([result[1] for result in final_results])
    class_found = final_syllabus[:-len('.txt')]

    return f'Here are class information for {full_course_info[class_found]}:\n{all_segments}'

In [ ]:
import requests

def general_news_report(query):
    """
    Fetches news articles based on the user's query.

    Args:
        query (str): The search query for the news topics.
        max_articles (int): Maximum number of articles to retrieve.

    Returns:
        str: A formatted string containing the titles and descriptions of the retrieved articles.
    """
    # Retrieve the News API key from user data or configuration
    new_api_key = userdata.get('new_api')  # Ensure this variable is correctly set in your environment
    max_articles=5
    # News API endpoint
    endpoint = 'https://newsapi.org/v2/everything'

    # Set query parameters
    params = {
        'q': query,                # Use the user's query
        'language': 'en',          # Language preference
        'sortBy': 'publishedAt',   # Order by most recent
        'apiKey': new_api_key,
    }

    # Send request to News API
    response = requests.get(endpoint, params=params)

    # Check for successful response
    if response.status_code == 200:
        news_data = response.json()
        articles = news_data.get('articles', [])[:max_articles]  # Limit the number of articles
        final_res = ""

        # Parse and append each article's title and description
        for article in articles:
            title = article.get('title', 'No Title')
            description = article.get('description', 'No Description')
            final_res += f"Title: {title}\n"
            final_res += f"Description: {description}\n\n"

        # Return the formatted string
        return final_res.strip()

    else:
        # Handle errors and return the response status
        error_message = f"Failed to retrieve news: {response.status_code} - {response.reason}"
        print(error_message)
        return error_message

In [ ]:
print(general_news_report("Los Angeles"))

Title: [Removed]
Description: [Removed]

Title: Nelly Knows “Where The Party At” with His 2025 World Tour
Description: Celebrating the 25th anniversary of his groundbreaking debut album, Country Grammar, music superstar and three-time Grammy winner Nelly is set to take the party global in 2025. The "Where The Party At Tour" promises over 50 electrifying dates across four cont…

Title: MISTER CARTOON x XLARGE Capsule Collection
Description: The legendary streetwear brand XLARGE has teamed with another street legend, MISTER CARTOON, on a special capsule collection that is available now. XLARGE was a pioneer of street culture in the 1990s, and CARTOON is a hybrid of so much of LA's culture, from s…

Title: Could Campus Activity Involvement Keep Students in College?
Description: Could Campus Activity Involvement Keep Students in College?

 


 
Melissa Ezarik

Tue, 12/10/2024 - 03:00 AM

Student Voice data looks at what the college experience was like for those who stopped out. Their lack 

In [ ]:
import json
import requests
from datetime import datetime, timedelta

def fetch_weather(params) -> str:
    """
    Fetch weather information based on the provided location and date.

    Args:
        params (str): A string containing the location and date in the format 'Location, Date'.

    Returns:
        str: A stringified dictionary containing the weather details or an error/warning message.
    """
    # Expecting params as a string, e.g., 'Los Angeles, December 5, 2024'
    if not isinstance(params, str):
        return "Error: Invalid input format. Expected a string 'Location, Date'."

    try:
        # Split the input into location and date
        location, date_str = map(str.strip, params.split(',', 1))
        # Parse the date into a standardized format
        query_date = datetime.strptime(date_str, "%B %d, %Y").date()
    except ValueError:
        return "Error: 'params' format is invalid. Expected format: 'Location, Month Day, Year'."

    # Validate the query date (ensure it is not beyond the forecast range)
    today = datetime.now().date()
    max_forecast_date = today + timedelta(days=3)  # wttr.in provides a 3-day forecast
    if query_date > max_forecast_date:
        return f"Warning: Weather data for {query_date} is not available. Please provide a date within the next 3 days."

    # Make the API call to wttr.in
    try:
        response = requests.get(f"https://wttr.in/{location}?format=j1", timeout=10)
        response.raise_for_status()
        weather_data = response.json()
    except requests.RequestException as e:
        return f"Error fetching weather data: {e}"

    # Handle current and forecast weather
    try:
        if query_date == today:
            # Return current weather
            current_condition = weather_data.get("current_condition", [{}])[0]
            result = {
                "FeelsLikeC": current_condition.get("FeelsLikeC", "N/A"),
                "temp_C": current_condition.get("temp_C", "N/A"),
                "weatherDesc": current_condition.get("weatherDesc", [{}])[0].get("value", "N/A"),
                "humidity": current_condition.get("humidity", "N/A"),
            }
            return f"Current weather for {location}: {result}"
        else:
            # Return forecast weather
            forecast = weather_data.get("weather", [])
            for day_forecast in forecast:
                forecast_date = datetime.strptime(day_forecast["date"], "%Y-%m-%d").date()
                if forecast_date == query_date:
                    hourly_data = day_forecast.get("hourly", [{}])
                    result = {
                        "max_temp": day_forecast.get("maxtempC", "N/A"),
                        "min_temp": day_forecast.get("mintempC", "N/A"),
                        "description": hourly_data[0].get("weatherDesc", [{}])[0].get("value", "N/A"),
                    }
                    return f"Forecast weather for {location} on {query_date}: {result}"
            return f"Weather forecast for {query_date} is not available. Please try another date."
    except KeyError as e:
        return f"Error processing weather data: Key {e} not found in the response."

In [ ]:
tools_list = [
    {
        "type": "function",
        "function": {
            "name": "RAG",
            "description": "Retrieve the relevant section in the given knowledge base when the user asks information about courses or syllabus.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The original question that the user asks exactly, no need to rephrase.",
                    },
                },
                "required": ["query"],
                "additionalProperties": False,
            },
        }
    },
     {
            "type": "function",
            "function": {
                "name": "fetch_weather",
                "description": "Fetches the current weather for a specified city with user-defined key selections.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "params": {
                            "type": "string",
                            "description": (
                                "A string containing: "
                                "the name of the city (e.g., 'London') and the Date of searching (e.g., December 5, 2024)"
                            ),
                        },
                    },
                    "required": ["params"],
                    "additionalProperties": False,
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "general_news_report",
                "description": "Fetches recent news articles based on the user's query.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The user's search query for the news (e.g., 'artificial intelligence').",
                        },
                    },
                    "required": ["query"],
                    "additionalProperties": False,
                },
            },
        },
]

In [ ]:
toolPrompt = f"""
You have access to the following functions:

Use the function '{tools_list[0]['function']['name']}' to '{tools_list[0]['function']['description']}'.
The parameters are: {json.dumps(tools_list[0]['function']['parameters']['properties'])}, where {tools_list[0]['function']['parameters']['required']} are required.

Use the function '{tools_list[1]['function']['name']}' to '{tools_list[1]['function']['description']}':
The parameters are: {json.dumps(tools_list[1]['function']['parameters']['properties'])}, where {tools_list[1]['function']['parameters']['required']} are required.

Use the function '{tools_list[2]['function']['name']}' to '{tools_list[2]['function']['description']}':
The parameters are: {json.dumps(tools_list[2]['function']['parameters']['properties'])}, where {tools_list[2]['function']['parameters']['required']} are required.

If you choose to call a function ONLY reply in the following format with no prefix or suffix:

<function=example_function_name>{{\"example_name\": \"example_value\"}}</function>

Reminder:
- Function calls MUST follow the specified format, start with <function= and end with </function>
- Required parameters MUST be specified
- You can call multiple if user need them
- Put the entire function call reply on one line
- If there is no function call available, answer the question like normal with your current knowledge and do not tell the user about function calls

"""

messages = [
  	{
        "role": "system",
        "content": toolPrompt,
    },
    {
        "role": "user",
        "content": query,
    },

]

response = together.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    messages=messages,
    max_tokens=1024,
    temperature=0,
    tools=tools_list,
    tool_choice="auto",
)

messages.append(response.choices[0].message)
print(response.choices[0].message.content)

None


In [ ]:
print(toolPrompt)


You have access to the following functions:

Use the function 'RAG' to 'Retrieve the relevant section in the given knowledge base when the user asks information about courses or syllabus.'.
The parameters are: {"query": {"type": "string", "description": "The original question that the user asks exactly, no need to rephrase."}}, where ['query'] are required.

Use the function 'fetch_weather' to 'Fetches the current weather for a specified city with user-defined key selections.':
The parameters are: {"params": {"type": "string", "description": "A string containing: the name of the city (e.g., 'London') and the Date of searching (e.g., December 5, 2024)"}}, where ['params'] are required.

Use the function 'general_news_report' to 'Fetches recent news articles based on the user's query.':
The parameters are: {"query": {"type": "string", "description": "The user's search query for the news (e.g., 'artificial intelligence')."}}, where ['query'] are required.

If you choose to call a functio

In [ ]:
print(response.choices[0].message)  # -> role=<MessageRole.ASSISTANT: 'assistant'> content=None tool_calls=[ToolCalls(id='call_5wbhh8zyrqa8hf98nw3lqfyu', type='function', function=FunctionCall(name='RAG', arguments='{"query":"csci585 final exam date"}'), index=0)]
# print(response.choices[0].message.tool_calls[0])  # one function at a time -> take index 0 -> id='call_5wbhh8zyrqa8hf98nw3lqfyu' type='function' function=FunctionCall(name='RAG', arguments='{"query":"csci585 final exam date"}') index=0
# print()
# print(response.choices[0].message.tool_calls[0].function)  # -> name='RAG' arguments='{"query":"csci585 final exam date"}'
# print()
# print(response.choices[0].message.tool_calls[0].function.name)  # Function name -> RAG
# print()
# print(response.choices[0].message.tool_calls[0].function.arguments)  # Function arguments -> {"query":"csci585 final exam date"}

role=<MessageRole.ASSISTANT: 'assistant'> content=None tool_calls=[ToolCalls(id='call_h3xojki7cuo8shdjrs326oap', type='function', function=FunctionCall(name='general_news_report', arguments='{"query":"Los Angeles news"}'), index=0)]


In [ ]:
import json
import re

def parse_tool_response(response_message):
    """
    Parses the tool response for function calls and arguments.

    Args:
        response_message: Response message object with content or tool_calls.

    Returns:
        dict or None: Parsed response with function name and arguments as a string,
                      or None if parsing fails.
    """
    # Check if tool_calls are already provided
    if hasattr(response_message, "tool_calls") and response_message.tool_calls:
        try:
            parsed_response = {
                "function": response_message.tool_calls[0].function.name,
                "arguments": response_message.tool_calls[0].function.arguments
            }
            return parsed_response
        except Exception as e:
            print(f"Error parsing tool_calls arguments: {e}")
            return None

    # Regex pattern to extract function calls and arguments
    function_regex = r"<function=([a-zA-Z_]\w*)>(\{.*?\})\s*(?:</function>|<function(?:/[\w]*)?>)"
    match = re.search(function_regex, response_message.content)

    if match:
        function_name, args_string = match.groups()
        print("Extracted function_name:", function_name)
        print("Extracted args_string:", args_string)

        # Simply return the function name and raw arguments as they are valid JSON strings
        return {
            "function": function_name,
            "arguments": args_string
        }

    return None


In [ ]:
parsed_response = parse_tool_response(response.choices[0].message)
print(response.choices[0].message)
if parsed_response is not None:
    print('The model decide to call a function: ')
    print(parsed_response)
else:
    print("No function call found in the response")
    print(response.choices[0].message.content)
print(parsed_response["arguments"])

role=<MessageRole.ASSISTANT: 'assistant'> content=None tool_calls=[ToolCalls(id='call_h3xojki7cuo8shdjrs326oap', type='function', function=FunctionCall(name='general_news_report', arguments='{"query":"Los Angeles news"}'), index=0)]
The model decide to call a function: 
{'function': 'general_news_report', 'arguments': '{"query":"Los Angeles news"}'}
{"query":"Los Angeles news"}


In [ ]:
if parsed_response:
    available_functions = {
        "RAG": RAG,
        "fetch_weather": fetch_weather,
        "general_news_report": general_news_report,
    }

    if parsed_response["function"] not in available_functions:
        available_function_names = "\n".join(available_functions.keys())
        raise NotImplementedError(
            f"Function {parsed_response['function']} is not implemented. "
            f"Our available functions are:\n\n{available_function_names}"
        )
    try:
        arguments = json.loads(parsed_response["arguments"])
    except json.JSONDecodeError as e:
        raise ValueError(f"Failed to parse arguments: {e}")

    function_to_call = available_functions[parsed_response["function"]]
    # print(function_to_call)
    # print(parsed_response["arguments"])
    # print(function_to_call(**arguments))
    print()

    result = function_to_call(**arguments)
    # print("function call result:", result)
    print()

    answer_prompt = "You are a very helpful assistant. Please answer user's question according to given information. Trust the given information, it is completely align with the user's question."

    new_messages = [
        {
            "role": "system",
            "content": answer_prompt,
        },
        {
            "role": "user",
            "content": f"""
                        ## Question:
                        {query}

                        ## Information:
                        {result}
                        """
        },
    ]

    res = together.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        messages=new_messages,
        max_tokens=1000,
        temperature=0.9,
    )
    print("Answer from the LLM: ", res.choices[0].message)
else:
    print("No function call found in the response")



Answer from the LLM:  role=<MessageRole.ASSISTANT: 'assistant'> content='According to the given information, there are several events happening in Los Angeles:\n\n1. Pacific Avenue Capital Partners, a private equity firm, has established an office in Paris, France, and has also designated Xavier Lambert as the Head of Europe, with operations set to expand into the EU, although the firm is headquartered in Los Angeles.\n\n2. There is no direct information about an "event" happening in Los Angeles, but rather Pacific Avenue Capital Partners having its headquarters in Los Angeles.' tool_calls=[]


In [ ]:
print(answer_prompt)
print(new_messages[1]['content'])

You are a very helpful assistant. Please answer user's question according to given information. Trust the given information, it is completely align with the user's question.

                        ## Question:
                        what is happening in Los Angeles?

                        ## Information:
                        Title: Oppenheimer Issues Positive Forecast for Porch Group (NASDAQ:PRCH) Stock Price
Description: Porch Group (NASDAQ:PRCH – Free Report) had its target price lifted by Oppenheimer from $4.00 to $7.00 in a research note released on Monday morning,Benzinga reports. They currently have an outperform rating on the stock. Other analysts have also issued resea…

Title: Pacific Avenue Capital Partners Establishes Paris France Office and Xavier Lambert Joins as Head of Europe to Lead Firm’s EU Expansion
Description: LOS ANGELES & PARIS — Pacific Avenue Capital Partners, LLC (“Pacific Avenue”), a Los Angeles-headquartered private equity firm focused on corporate d

In [ ]:
print(res.choices[0].message.content)

According to the given information, there are several events happening in Los Angeles:

1. Pacific Avenue Capital Partners, a private equity firm, has established an office in Paris, France, and has also designated Xavier Lambert as the Head of Europe, with operations set to expand into the EU, although the firm is headquartered in Los Angeles.

2. There is no direct information about an "event" happening in Los Angeles, but rather Pacific Avenue Capital Partners having its headquarters in Los Angeles.


In [ ]:
def main():
    previous_RAG_info = None  # Placeholder for RAG information from prior turns
    conversation_count = 0
    chat_history = []  # To store the latest 10 conversations

    while True:
        # Get user query
        user_query = input("User: ")

        if user_query.lower() in ["exit", "quit"]:
            print("Exiting conversation.")
            break
        use_previous_RAG = False
        # Decide if previous RAG info should be used (dummy condition for now)
        if conversation_count > 0 and previous_RAG_info:
            use_previous_RAG = False  # Replace with actual condition
        else:
            use_previous_RAG = False

        if use_previous_RAG:
            user_query = f"{user_query}\n\nAdditional context:\n{previous_RAG_info}"

        # Add the new user query to the chat history
        chat_history.append({"role": "user", "content": user_query})

        # Ensure the chat history is limited to the latest 10 exchanges
        if len(chat_history) > 20:  # Each "exchange" is a user+assistant pair
            chat_history = chat_history[-20:]

        # Construct messages with chat history
        messages = [{"role": "system", "content": toolPrompt}] + chat_history

        # Call the LLM
        response = together.chat.completions.create(
            model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
            messages=messages,
            max_tokens=1024,
            temperature=0,
            tools=tools_list,
            tool_choice="auto",
        )

        # Parse the response
        response_message = response.choices[0].message
        parsed_response = parse_tool_response(response_message)

        # Add assistant's response to chat history
        if parsed_response:
            available_functions = {
                "RAG": RAG,
                "fetch_weather": fetch_weather,
                "general_news_report": general_news_report,
            }
            function_name = parsed_response["function"]

            arguments = json.loads(parsed_response["arguments"])
            if function_name in available_functions:
                result = available_functions[function_name](**arguments) #应该是车越的意思
                if function_name == "RAG":
                    previous_RAG_info = result  # Store RAG info for future turns
                    result += "Location: Los Angeles"
            else:
                print(f"Unknown function: {function_name}")

            # Prepare the answer prompt for the second LLM call
            answer_prompt = (
                "You are a very helpful assistant. Please answer the user's question using the information provided below. "
                "Trust the given information, as it is aligned with the user's query.\n\n"
                f"## Question:\n{user_query}\n\n"
                f"## Information:\n{result}"
            )

            # Create new LLM messages
            new_messages = [{"role": "system", "content": "You are a helpful assistant."}]
            new_messages += chat_history
            new_messages.append({"role": "user", "content": answer_prompt})

            # Call the LLM again with the tool's result
            final_response = together.chat.completions.create(
                model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
                messages=new_messages,
                max_tokens=1000,
                temperature=0.9,
            )
            assistant_response = final_response.choices[0].message.content
            print("Agent:", assistant_response)
            chat_history.append({"role": "assistant", "content": assistant_response})
        else:
            messages = [{"role": "system", "content": "You are a helpful assistant."}]
            messages += chat_history
            messages.append({"role": "user", "content": user_query})

            # Call the LLM to generate a response
            response = together.chat.completions.create(
                model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
                messages=messages,
                max_tokens=1000,
                temperature=0.9,
            )
            assistant_response = response.choices[0].message.content
            print("Agent:", assistant_response)
            chat_history.append({"role": "assistant", "content": assistant_response})

        print()
        print("**********split**********")
        print()

        # Increment conversation count
        conversation_count += 1


In [ ]:
main()

User: quit
Exiting conversation.


In [ ]:
def main_MultiFunction():
    previous_RAG_info = None  # Placeholder for RAG information from prior turns
    conversation_count = 0
    chat_history = []  # To store the latest 10 conversations

    while True:
        # Get user query
        user_query = input("User: ")

        if user_query.lower() in ["exit", "quit"]:
            print("Exiting conversation.")
            break
        use_previous_RAG = False
        # Decide if previous RAG info should be used (dummy condition for now)
        if conversation_count > 0 and previous_RAG_info:
            use_previous_RAG = False  # Replace with actual condition
        else:
            use_previous_RAG = False

        if use_previous_RAG:
            user_query = f"{user_query}\n\nAdditional context:\n{previous_RAG_info}"

        # Add the new user query to the chat history
        chat_history.append({"role": "user", "content": user_query})

        # Ensure the chat history is limited to the latest 10 exchanges
        if len(chat_history) > 20:  # Each "exchange" is a user+assistant pair
            chat_history = chat_history[-20:]

        # Construct messages with chat history
        messages = [{"role": "system", "content": toolPrompt}] + chat_history

        # Call the LLM
        response = together.chat.completions.create(
            model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
            messages=messages,
            max_tokens=1024,
            temperature=0,
            tools=tools_list,
            tool_choice="auto",
        )

        # Parse the response
        response_message = response.choices[0].message
        parsed_response = parse_tool_response(response_message)
        print("parsed_response",parsed_response)
        # Add assistant's response to chat history
        if parsed_response:
            available_functions = {
                "RAG": RAG,
                "fetch_weather": fetch_weather,
                "general_news_report": general_news_report,
            }
            tools_to_process = []  # A list to handle sequential tool calls
            arguments = None
            result = None
            while_out = 0
            # Handle the first function and queue the next if necessary
            while parsed_response:
                while_out += 1
                if while_out > 1:
                    break
                function_name = parsed_response["function"]
                print(function_name,"function name")
                arguments = json.loads(parsed_response["arguments"])
                print(arguments,"arguments")
                if function_name in available_functions:
                    result = available_functions[function_name](**arguments)
                    if function_name == "RAG":
                        previous_RAG_info = result  # Store RAG info for future turns
                        result += "END RAG RESULT\tLocation: Los Angeles \t"
                    tools_to_process.append({"function": function_name, "result": result})

                    # If more tools are needed, ask the agent to infer the next step
                    new_tool_prompt = (

                        "Based on the result of the tool call, determine if additional functions are needed. Do not Repeat Function Call! "
                        "If needed, provide the function name and arguments and call the function again thank you!"
                    )
                    next_tool_messages = [{"role": "system", "content": new_tool_prompt}]
                    next_tool_messages.append({"role": "user", "content": f"You have used {tools_to_process[-1]}, and result for it is {result}" + f" Do not Repeat Function Call! This is user query again : {user_query}. This is Tool use list: {toolPrompt}"})

                    next_tool_response = together.chat.completions.create(
                        model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
                        messages=next_tool_messages,
                        max_tokens=500,
                        tools=tools_list,
                        temperature=0,
                    )
                    # print(next_tool_messages,"prompt")
                    parsed_response = parse_tool_response(next_tool_response.choices[0].message)

                    # print(parsed_response,"inside while loop")
                else:
                    print(f"Unknown function: {function_name}")
                    break

            # Use the final tool result to generate the user-facing response
            final_result_summary = "\n".join(
                [f"Tool: {tool['function']}\nResult: {tool['result']}" for tool in tools_to_process]
            )
            final_response_prompt = (
                "Based on the following tool results, provide a helpful answer to the user's query:\n\n"
                f"{final_result_summary}\n\nUser query:\n{user_query}"
            )

            # Create final response
            final_messages = [{"role": "system", "content": "You are a helpful assistant. Below is a series of history conversation"}]
            final_messages += chat_history
            final_messages.append({"role": "user", "content": final_response_prompt})

            final_response = together.chat.completions.create(
                model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
                messages=final_messages,
                max_tokens=1000,
                temperature=0.9,
            )
            assistant_response = final_response.choices[0].message.content
            print("Agent:", assistant_response)
            chat_history.append({"role": "assistant", "content": assistant_response})
        else:
            # Direct LLM response
            messages = [{"role": "system", "content": "You are a helpful assistant."}]
            messages += chat_history
            messages.append({"role": "user", "content": user_query})

            # Call the LLM to generate a response
            response = together.chat.completions.create(
                model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
                messages=messages,
                max_tokens=1000,
                temperature=0.9,
            )
            assistant_response = response.choices[0].message.content
            print("Agent:", assistant_response)
            chat_history.append({"role": "assistant", "content": assistant_response})

        print()
        print("**********split**********")
        print()

        # Increment conversation count
        conversation_count += 1


In [ ]:
main_MultiFunction()

User: Hi I wonder for the CSCI 544 when is the final exam date? Also, if the day going to be a sunny day?
parsed_response None
Agent: I'd be happy to help you with that, but I'm a large language model, I don't have real-time access to specific information about your university or course schedule.

However, I can suggest a few possible ways to find out the final exam date for CSCI 544:

1. Check the university's course catalog or website: You can typically find course schedules and exam dates in the course catalog or on the university's website.
2. Contact the department: You can reach out to the Computer Science department at your university and ask for the final exam date for CSCI 544.
3. Check with your instructor: If you have your instructor's contact information, you can reach out to them directly and ask for the final exam date.

As for the weather, I can provide you with general information about sunny days, but I won't be able to predict the weather for a specific location or da

In [ ]:
def main_MultiFunction():
    previous_RAG_info = None  # Placeholder for RAG information from prior turns
    conversation_count = 0
    chat_history = []  # To store the latest 10 conversations
    executed_calls = set()  # Track executed function calls

    while True:
        # Get user query
        user_query = input("User: ")

        if user_query.lower() in ["exit", "quit"]:
            print("Exiting conversation.")
            break

        use_previous_RAG = False
        # Decide if previous RAG info should be used (dummy condition for now)
        if conversation_count > 0 and previous_RAG_info:
            use_previous_RAG = False  # Replace with actual condition
        else:
            use_previous_RAG = False

        if use_previous_RAG:
            user_query = f"{user_query}\n\nAdditional context:\n{previous_RAG_info}"

        # Add the new user query to the chat history
        chat_history.append({"role": "user", "content": user_query})

        # Ensure the chat history is limited to the latest 10 exchanges
        if len(chat_history) > 20:  # Each "exchange" is a user+assistant pair
            chat_history = chat_history[-20:]

        # Construct messages with chat history
        messages = [{"role": "system", "content": toolPrompt}] + chat_history

        # Call the LLM
        response = together.chat.completions.create(
            model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
            messages=messages,
            max_tokens=1024,
            temperature=0,
            tools=tools_list,
            tool_choice="auto",
        )

        # Parse the response
        response_message = response.choices[0].message
        parsed_response = parse_tool_response(response_message)
        print("parsed_response", parsed_response)

        if parsed_response:
            available_functions = {
                "RAG": RAG,
                "fetch_weather": fetch_weather,
                "general_news_report": general_news_report,
            }
            tools_to_process = []  # A list to handle sequential tool calls
            result = None

            while_out = 0
            while parsed_response:
                while_out += 1
                if while_out > 3:  # Prevent infinite loops
                    print("Breaking loop due to excessive iterations.")
                    break

                function_name = parsed_response["function"]
                arguments = json.dumps(parsed_response["arguments"])  # Serialize arguments to track as a key

                if (function_name, arguments) in executed_calls:
                    # print(f"Skipping repeated function call: {function_name} with arguments: {arguments}")
                    break  # Prevent repeated calls

                executed_calls.add((function_name, arguments))  # Add to executed calls
                # print(function_name, "function name")
                # print(arguments, "arguments")

                if function_name in available_functions:
                    result = available_functions[function_name](**json.loads(arguments))
                    if function_name == "RAG":
                        previous_RAG_info = result  # Store RAG info for future turns
                        result += "END RAG RESULT\tLocation: Los Angeles \t"
                    tools_to_process.append({"function": function_name, "result": result})

                    # Determine next tool call
                    new_tool_prompt = (
                        "Based on the result of the tool call, determine if additional functions are needed. Do not Repeat Function Call! "
                        "If needed, provide the function name and arguments and call the function again thank you!"
                    )
                    next_tool_messages = [{"role": "system", "content": new_tool_prompt}]
                    next_tool_messages += chat_history
                    next_tool_messages.append({"role": "user", "content": f"tool_result: {result}"})

                    next_tool_response = together.chat.completions.create(
                        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
                        messages=next_tool_messages,
                        max_tokens=500,
                        tools=tools_list,
                        temperature=0,
                    )
                    # print(next_tool_messages, "prompt")
                    next_tool_parsed = parse_tool_response(next_tool_response.choices[0].message)
                    parsed_response = next_tool_parsed
                    # print(parsed_response, "inside while loop")
                else:
                    print(f"Unknown function: {function_name}")
                    break

            # Generate user-facing response
            final_result_summary = "\n".join(
                [f"Tool: {tool['function']}\nResult: {tool['result']}" for tool in tools_to_process]
            )
            final_response_prompt = (
                "Based on the following tool results, provide a helpful answer to the user's query:\n\n"
                f"{final_result_summary}\n\nUser query:\n{user_query}"
            )

            final_messages = [{"role": "system", "content": "You are a helpful assistant. Below is a series of history conversation"}]
            final_messages += chat_history
            final_messages.append({"role": "user", "content": final_response_prompt})

            final_response = together.chat.completions.create(
                model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
                messages=final_messages,
                max_tokens=1000,
                temperature=0.9,
            )
            assistant_response = final_response.choices[0].message.content
            print("Agent:", assistant_response)
            chat_history.append({"role": "assistant", "content": assistant_response})
        else:
            # Direct response
            messages = [{"role": "system", "content": "You are a helpful assistant."}]
            messages += chat_history
            messages.append({"role": "user", "content": user_query})

            response = together.chat.completions.create(
                model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
                messages=messages,
                max_tokens=1000,
                temperature=0.9,
            )
            assistant_response = response.choices[0].message.content
            print("Agent:", assistant_response)
            chat_history.append({"role": "assistant", "content": assistant_response})

        print("\n**********split**********\n")
        conversation_count += 1


In [ ]:
print(fetch_weather('Los Angeles, December 5, 2024'))


Weather forecast for 2024-12-05 is not available. Please try another date.


In [ ]:
print(fetch_weather('Los Angeles, December 11, 2024'))

Forecast weather for Los Angeles on 2024-12-11: {'max_temp': '22', 'min_temp': '10', 'description': 'Clear '}
